In [61]:
import pandas as pd  # Importa la biblioteca pandas.

# Cargar datos de prueba desde un archivo zip.
test_data = pd.read_csv(
    "../files/input/test_data.csv.zip",  # Especifica la ruta relativa al archivo .zip que contiene los datos de prueba.
    index_col=False,  # Indica que no se debe usar ninguna columna como índice del DataFrame.
    compression="zip",  # Especifica que el archivo está comprimido como un .zip.
)

# Cargar datos de entrenamiento desde un archivo zip.
train_data = pd.read_csv(
    "../files/input/train_data.csv.zip",  # Especifica la ruta relativa al archivo .zip que contiene los datos de entrenamiento.
    index_col=False,  # Indica que no se debe usar ninguna columna como índice del DataFrame.
    compression="zip",  # Especifica que el archivo está comprimido como un .zip.
)

In [62]:
#
# limpieza de los datos
#

import numpy as np # importa la biblioteca numpy

# Renombrar la columna 'default payment next month' a 'default'.
test_data.rename(columns={"default payment next month": "default"}, inplace=True)
train_data.rename(columns={"default payment next month": "default"}, inplace=True)

# Eliminar la columna 'ID' de ambos DataFrames.
test_data.drop(columns='ID', inplace=True)
train_data.drop(columns='ID', inplace=True)

# Filtrar y eliminar filas donde los registros con informacion no disponible.
test_data = test_data[(test_data['EDUCATION'] != 0) & (test_data['MARRIAGE'] != 0)]
train_data = train_data[(train_data['EDUCATION'] != 0) & (train_data['MARRIAGE'] != 0)]

# Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".
test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

In [63]:
#
# separacion de los datos
#

# Elimina la columna 'default' del DataFrame train_data para crear el conjunto de características de entrenamiento.
x_train = train_data.drop(columns="default")

# Extrae la columna 'default' de train_data y la usa como el conjunto de etiquetas de entrenamiento.
y_train = train_data["default"]

# Elimina la columna 'default' del DataFrame test_data para crear el conjunto de características de prueba.
x_test = test_data.drop(columns="default")

# Extrae la columna 'default' de test_data y la usa como el conjunto de etiquetas de prueba.
y_test = test_data["default"]

In [64]:
#
# eleccion modelo y transformaciones
#

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

# Definición de las columnas categóricas y numéricas.
categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]  # Lista de columnas categóricas.
numeric_features = x_train.columns.difference(categorical_features + ["default"]).tolist()  # Lista de columnas numéricas.

# Configuración del preprocesador.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),  # Codifica variables categóricas.
        ('scaler', StandardScaler(), numeric_features)  # Escala las variables numéricas para tener media 0 y desviación estándar 1.
    ],
    remainder='passthrough'  # Pasa sin cambios las columnas que no están incluidas en 'transformers'.
)

# Creación del pipeline.
pipeline = Pipeline([
    ("preprocessor", preprocessor),  # Aplica las transformaciones del preprocesador.
    ('pca', PCA()),  # Aplica PCA para reducir la dimensionalidad.
    ("feature_selection", SelectKBest(score_func=f_classif)),  # Selecciona las características más relevantes.
    ("classifier", SVC(kernel="rbf", random_state=0, max_iter=-1))  # Usa un modelo de SVM con kernel RBF.
])

In [65]:
#
# hiperparametros y ajuste
#

# Definición de la rejilla de parámetros para la búsqueda.
param_grid = {
    'pca__n_components': [20, x_train.shape[1]-2],  # Número de componentes principales a mantener.
    'feature_selection__k': [12],  # Número de características a seleccionar.
    'classifier__gamma': [0.1]  # Parámetro gamma para el kernel RBF.
}

# Configuración de GridSearchCV.
model = GridSearchCV(
    estimator=pipeline,  # El pipeline que incluye preprocesamiento, selección de características y clasificador.
    param_grid=param_grid,  # La rejilla de hiperparámetros definida arriba.
    scoring='balanced_accuracy',  # Usa balanced_accuracy como métrica de evaluación.
    cv=10,  # Realiza validación cruzada con 10 particiones.
    n_jobs=-1  # Usa todos los núcleos disponibles para procesar las combinaciones en paralelo.
)

# Ajusta el modelo en el conjunto de entrenamiento.
model.fit(x_train, y_train)

# Imprimir los mejores parámetros y el mejor score.
print("Mejores parámetros:", model.best_params_)
print("Mejor precisión balanceada:", model.best_score_)

Mejores parámetros: {'classifier__gamma': 0.1, 'feature_selection__k': 12, 'pca__n_components': 20}
Mejor precisión balanceada: 0.6500332144162928


Mejores parámetros: {'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'feature_selection__k': 12, 'pca__n_components': 20}

Mejor precisión balanceada: 0.6500332144162928

In [66]:
#
# guardar modelo
#

import gzip  # Para comprimir y descomprimir archivos.
import pickle  # Para serializar y deserializar objetos de Python, como modelos de machine learning.
import os  # Para manejo de directorios.

models_dir = '../files/models'  # Define la ruta donde se guardarán los modelos.
os.makedirs(models_dir, exist_ok=True)  # Crea el directorio si no existe.

model_path = "../files/models/model.pkl.gz"  # Define la ruta completa para el archivo del modelo.

with gzip.open(model_path, "wb") as f:  # Abre el archivo en modo escritura binaria ('wb') y comprime el contenido.
    pickle.dump(model, f)  # Serializa el objeto del modelo y lo guarda en el archivo comprimido.

model_path  # Devuelve la ruta donde se guardó el modelo.

'../files/models/model.pkl.gz'

In [67]:
#
# funcion para guardar  metricas en json
#

import json  # Manejo de archivos JSON.
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score  # Métricas de evaluación.

def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
    
    y_train_pred = model.predict(X_train)  # Predice etiquetas para el conjunto de entrenamiento.
    y_test_pred = model.predict(X_test)  # Predice etiquetas para el conjunto de prueba.

    metrics_train = {
        'type': 'metrics',  # Tipo de datos guardados.
        'dataset': 'train',  # Identifica que las métricas corresponden al conjunto de entrenamiento.
        'precision': precision_score(y_train, y_train_pred, zero_division=0),  # Precisión del modelo.
        'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),  # Exactitud balanceada.
        'recall': recall_score(y_train, y_train_pred, zero_division=0),  # Sensibilidad o recall.
        'f1_score': f1_score(y_train, y_train_pred, zero_division=0)  # Puntaje F1.
    }

    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',  # Identifica que las métricas corresponden al conjunto de prueba.
        'precision': precision_score(y_test, y_test_pred, zero_division=0),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred, zero_division=0),
        'f1_score': f1_score(y_test, y_test_pred, zero_division=0)
    }

    output_dir = os.path.abspath('../files/output')  # Define el directorio donde se guardarán los resultados.
    os.makedirs(output_dir, exist_ok=True)  # Crea el directorio si no existe.

    output_path = os.path.join(output_dir, 'metrics.json')  # Define la ruta completa para el archivo JSON.
    with open(output_path, 'w') as f:  # Abre el archivo en modo escritura.
        f.write(json.dumps(metrics_train) + '\n')  # Escribe las métricas del conjunto de entrenamiento en formato JSON.
        f.write(json.dumps(metrics_test) + '\n')  # Escribe las métricas del conjunto de prueba en formato JSON.

    return metrics_train, metrics_test # Retornar las métricas

In [68]:
#
# funcion para guardar matrices de confucion en json
#

from sklearn.metrics import confusion_matrix  # Importa la función para calcular matrices de confusión.

def calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test):

    # Hacer predicciones
    y_train_pred = model.predict(X_train)  # Predice etiquetas para el conjunto de entrenamiento.
    y_test_pred = model.predict(X_test)  # Predice etiquetas para el conjunto de prueba.

    # Calcular matrices de confusión
    cm_train = confusion_matrix(y_train, y_train_pred)  # Calcula la matriz de confusión para el conjunto de entrenamiento.
    cm_test = confusion_matrix(y_test, y_test_pred)  # Calcula la matriz de confusión para el conjunto de prueba.

    # Convertir las matrices de confusión en formato JSON
    def format_confusion_matrix(cm, dataset_type):
        return {
            'type': 'cm_matrix',  # Tipo de métrica guardada.
            'dataset': dataset_type,  # Indica si es entrenamiento o prueba.
            'true_0': {  # Valores verdaderos para la clase 0.
                'predicted_0': int(cm[0, 0]),  # Predicción correcta para clase 0.
                'predicted_1': int(cm[0, 1])   # Predicción incorrecta para clase 0.
            },
            'true_1': {  # Valores verdaderos para la clase 1.
                'predicted_0': int(cm[1, 0]),  # Predicción incorrecta para clase 1.
                'predicted_1': int(cm[1, 1])   # Predicción correcta para clase 1.
            }
        }

    metrics = [
        format_confusion_matrix(cm_train, 'train'),  # Matriz de confusión para entrenamiento.
        format_confusion_matrix(cm_test, 'test')    # Matriz de confusión para prueba.
    ]

    # Guardar las matrices de confusión en el archivo JSON
    output_path = '../files/output/metrics.json'  # Ruta del archivo JSON.
    with open(output_path, 'a') as f:  # Abre el archivo en modo agregar ('a').
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')  # Convierte el diccionario en JSON y lo guarda.

    return cm_train, cm_test # Devuelve las matrices de confusión originales.

In [69]:
#
# funcion ejecutura
#

def main(model, X_train, X_test, y_train, y_test):

    # Crear el directorio de salida si no existe
    import os
    os.makedirs('../files/output', exist_ok=True)

    # Calcular y guardar las métricas
    metrics_train, metrics_test = calculate_and_save_metrics(model, X_train, X_test, y_train, y_test)

    # Imprimir las métricas
    print("Métricas del conjunto de entrenamiento:")
    for key, value in metrics_train.items():
        print(f"{key}: {value}")

    print("\nMétricas del conjunto de prueba:")
    for key, value in metrics_test.items():
        print(f"{key}: {value}")

    # Calcular, guardar e imprimir las matrices de confusión
    cm_train, cm_test = calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test)

    print("\nMatriz de confusión del conjunto de entrenamiento:")
    print(cm_train)

    print("\nMatriz de confusión del conjunto de prueba:")
    print(cm_test)

In [70]:
# Ejemplo de uso:
main(model, x_train, x_test, y_train, y_test)

Métricas del conjunto de entrenamiento:
type: metrics
dataset: train
precision: 0.7026920031670626
balanced_accuracy: 0.6646916688511463
recall: 0.37566137566137564
f1_score: 0.4895876430837126

Métricas del conjunto de prueba:
type: metrics
dataset: test
precision: 0.6736745886654479
balanced_accuracy: 0.6681000149987337
recall: 0.38667366211962223
f1_score: 0.49133333333333334

Matriz de confusión del conjunto de entrenamiento:
[[15477   751]
 [ 2950  1775]]

Matriz de confusión del conjunto de prueba:
[[6716  357]
 [1169  737]]
